<a href="https://colab.research.google.com/github/Mohamed-derbeli/Fuel-cell-system/blob/main/Modeling_of_a_Commercial_Heliocentris_PEMFC_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
import os
from os import getcwd
import urllib.request
# from urllib.request import urlopen
from urllib.request import urlretrieve
from tensorflow import keras
import codecs

In [ ]:
# with urllib.request.urlopen('https://raw.githubusercontent.com/Mohamed-derbeli/Fuel-cell-system/main/FC50_All_data00.csv') as csvfile:
#    read_data = csvfile.read().decode('utf-8')
#    print(read_data[0:50])

In [10]:
#Inputs/ Fan: Fan power, Temp: Temperature, I_stack: Stack current, 
#Outputs/ V_stack: Stack voltage

V_stack=[]
I_stack=[]
Temp=[]
H_2=[]
Fan=[]
P_stack=[]

# The data of each parapmeter in the scv file is divided into 10 rows. Therefoe, to obtain the whole data of each stack parameters, the data should be appended in the same vector. 
V_stack_index=[]
I_stack_index=[]
Temp_index=[]
H_2_index=[]
Fan_index=[]
P_stack_index=[]

with urllib.request.urlopen('https://raw.githubusercontent.com/Mohamed-derbeli/Fuel-cell-system/main/FC50_All_data00.csv') as csvfile:
    read_file = csv.reader(codecs.iterdecode(csvfile, 'utf-8'), delimiter=',')
    # next(read_file) # next() used to skip the first row 

    for i in range(0,6,1):  
        for number in range(i, 60, 6):
          if i==0:
            V_stack_index.append(number)
          if i==1:
            I_stack_index.append(number)
          if i==2:
            Temp_index.append(number)
          if i==3:
            H_2_index.append(number)
          if i==4:
            Fan_index.append(number)
          if i==5:
            P_stack_index.append(number)
    print("V_stack_index = ", V_stack_index)
    print("I_stack_index = ",I_stack_index)
    print("Temp_index = ",Temp_index)
    print("H_2_index = ",H_2_index)
    print("Fan_index = ",Fan_index)
    print("P_stack_index = ",P_stack_index)
    counter=0    
    for row in read_file:
      #clean the empty cells
      j=0
      for item in row:
        j=j+1
        if item =="":
          row=row[0:j-1]
      # print(j)
      #append the rows of the same parameter
      if counter in V_stack_index:
        V_stack.append(row)        
      if counter in I_stack_index:
        I_stack.append(row)
      if counter in Temp_index:
        Temp.append(row)
      if counter in H_2_index:
        H_2.append(row)
      if counter in Fan_index:
        Fan.append(row)
      if counter in P_stack_index:
        P_stack.append(row)
      counter=counter+1
    print("\n V_stack = ",V_stack[0:2][0:])
    print(len(V_stack[0]))
    print("I_stack= ",I_stack[0][0:10])
    print("Temp = ",Temp[0][0:10])
    print("H_2 = ", H_2[0][0:10])
    print("Fan = ", Fan[0][0:10])
    print("P_stack = ", P_stack[0][0:10])

        # labels.append(row[0])
        # sentence=row[1]
        # # for word in stopwords:
        # #     token= " " + word + " " # we used " " to add space before and after the word
        # #     sentence=sentence.replace(token," ") #remove the stopwords from the senctencs
        # #     sentence=sentence.replace("  "," ") #when there are 2 successively changes, they will cause 2 spaces and these should be removed
        # sentences.append(sentence)
        # read_via_pandas= pd.read_csv(csvfile)

V_stack_index =  [0, 6, 12, 18, 24, 30, 36, 42, 48, 54]
I_stack_index =  [1, 7, 13, 19, 25, 31, 37, 43, 49, 55]
Temp_index =  [2, 8, 14, 20, 26, 32, 38, 44, 50, 56]
H_2_index =  [3, 9, 15, 21, 27, 33, 39, 45, 51, 57]
Fan_index =  [4, 10, 16, 22, 28, 34, 40, 46, 52, 58]
P_stack_index =  [5, 11, 17, 23, 29, 35, 41, 47, 53, 59]

 V_stack =  [['6.45', '6.09', '5.34', '4.41', '4.61', '5.52', '6.04', '6.52', '6.93', '7.16', '7.35', '7.49', '7.61', '7.7', '7.79', '7.85', '7.91', '7.97', '8.06', '8.07', '7.95', '7.89', '7.81', '7.73', '7.66', '7.56', '7.42', '7.26', '7.06', '6.81', '6.48', '6.02', '5.39', '4.47', '4.65', '5.56', '6.19', '6.64', '6.96', '7.18', '7.32', '7.47', '7.61', '7.71', '7.79', '7.85', '7.92', '7.97', '8.06', '8.06', '7.95', '7.9', '7.81', '7.74', '7.65', '7.54', '7.43', '7.27', '7.07', '6.83', '6.5', '6.15', '5.41', '4.48', '4.68', '5.58', '6.19', '6.64', '6.95', '7.14', '7.36', '7.5', '7.61', '7.7', '7.79', '7.85', '7.9', '7.97', '8.07', '8.07', '7.97', '7.9', '7.85', '

In [3]:
# !gdown "https://github.com/Mohamed-derbeli/Fuel-cell-system/blob/main/FC50_All_data00.csv"
# Get the current working directory
# directory_path = os.getcwd()
# print("Current working directory: ", directory_path)

Downloading...
From: https://github.com/Mohamed-derbeli/Fuel-cell-system/blob/main/FC50_All_data00.csv
To: /content/FC50_All_data00.csv
134kB [00:00, 17.7MB/s]


In [ ]:
#  with open('./FC50_All_data00.csv','r') as csvfile:
# #   read_data =  pd.read_excel(exelfile)
#     read_data = csv.reader(csvfile, delimiter=',')
#     next(read_data)
#     print(read_data)
#     I=[]
#     for row in read_data:
#       I.append(row)
#     print(I)



  # Load the xlsx file
#excel_data = pd.read_excel('./FC50_All_data00.xlsx')
# Read the values of the file in the dataframe
# data = pd.DataFrame(excel_data, columns=[])
# # Print the content
# print("The content of the file is:\n", data)

FileNotFoundError: ignored

In [ ]:
#path= f"{getcwd()}/../tmp2/FC50_All_data00.xlsx"

# Get the current working directory
directory_path = os.getcwd()
print("Current working directory: ", directory_path)
# Change the current working directory
# os.chdir('/tmp')
# print("Current working directory: ", directory_path)

#read_data = open(directory_path) 
# pd.read_excel (path)



Current working directory:  /content
